In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import os

In [ ]:
import tempfile
import time
import torch
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForCausalLM
#from evaluate import load
import shutil

# Afinamiento

In [ ]:
MODEL_NAME = "codellama/CodeLlama-7b-hf"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")

Usando dispositivo: cuda


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
def load_jsonl_dataset(file_path):
    """Carga el dataset desde un archivo JSONL"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def create_prompt(source_code, target_code=None):
    """Crea el prompt para el entrenamiento"""
    if target_code:
        prompt = f"""Traduce el siguiente código de C a C++:

Código C:
{source_code}

Código C++:
{target_code}"""
    else:
        prompt = f"""Traduce el siguiente código de C a C++:

Código C:
{source_code}

Código C++:"""

    return prompt

In [ ]:
def preprocess_dataset(data, tokenizer, max_length=512):
    """Preprocesa el dataset para el entrenamiento"""
    texts = []

    for item in data:
        if all(key in item for key in ['source_code', 'target_code']):
            prompt = create_prompt(item['source_code'], item['target_code'])
            texts.append(prompt)

    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )

    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': encodings['input_ids'].clone()
    })

def setup_model_and_tokenizer():
    """Configura el modelo y tokenizer"""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16
    )

    return model, tokenizer

def setup_lora_config():
    """Configura LoRA para fine-tuning eficiente"""
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=16,  # Rank
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=[
            "q_proj",
            "v_proj",
            "k_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ]
    )
    return lora_config

In [ ]:
DATASET_PATH = "/content/train.jsonl"
OUTPUT_DIR = "./codellama-c-to-cpp-finetuned"

if not os.path.exists(DATASET_PATH):
  print(f"Error: No se encontró el archivo {DATASET_PATH}")
  print("Asegúrate de que el archivo existe y la ruta es correcta")
  exit(1)

In [ ]:
print("Cargando modelo y tokenizer...")
model, tokenizer = setup_model_and_tokenizer()

Cargando modelo y tokenizer...


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
print("Preparando modelo para entrenamiento...")
model = prepare_model_for_kbit_training(model)

print("Configurando LoRA...")
lora_config = setup_lora_config()
model = get_peft_model(model, lora_config)

print("Cargando y procesando dataset...")
raw_data = load_jsonl_dataset(DATASET_PATH)
print(f"Cargadas {len(raw_data)} muestras")

Preparando modelo para entrenamiento...
Configurando LoRA...
Cargando y procesando dataset...
Cargadas 3305 muestras


In [ ]:
split_idx = int(0.9 * len(raw_data))
train_data = raw_data[:split_idx]
val_data = raw_data[split_idx:]

train_dataset = preprocess_dataset(train_data, tokenizer)
val_dataset = preprocess_dataset(val_data, tokenizer)

print(f"Dataset de entrenamiento: {len(train_dataset)} muestras")
print(f"Dataset de validación: {len(val_dataset)} muestras")

Dataset de entrenamiento: 2974 muestras
Dataset de validación: 331 muestras


In [ ]:

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=10,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=50,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        greater_is_better=False,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        optim="adamw_torch",
        report_to="none"
    )

In [ ]:

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("Iniciando entrenamiento...")
trainer.train()

Iniciando entrenamiento...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,0.294297


TrainOutput(global_step=47, training_loss=0.3702923389191323, metrics={'train_runtime': 3269.9863, 'train_samples_per_second': 0.909, 'train_steps_per_second': 0.014, 'total_flos': 6.073137038465434e+16, 'train_loss': 0.3702923389191323, 'epoch': 1.0})

In [ ]:
print("Guardando modelo...")
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Modelo guardado en: {OUTPUT_DIR}")

Guardando modelo...
Modelo guardado en: ./codellama-c-to-cpp-finetuned


Para guardar el modelo en Hugging Face, necesitas un token de autenticación con permisos de escritura. Puedes obtener uno en tu página de configuración de Hugging Face: [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens).

Una vez que tengas el token, guárdalo de forma segura en los secretos de Colab (el icono de la llave 🔑 a la izquierda) con el nombre `HF_TOKEN`.

In [ ]:
from huggingface_hub import login

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    if HF_TOKEN is None:
        print("Error: HF_TOKEN no encontrado en los secretos de Colab.")
    else:
        login(token=HF_TOKEN)
        print("Inicio de sesión en Hugging Face exitoso.")
except ImportError:
    print("No estás en un entorno Colab o no se pudo acceder a userdata.")

Inicio de sesión en Hugging Face exitoso.


Ahora puedes guardar el modelo en un repositorio de Hugging Face. Reemplaza `your_username/your_model_name` con tu nombre de usuario de Hugging Face y el nombre que deseas para tu repositorio.

In [ ]:
from huggingface_hub import HfApi
import os

OUTPUT_DIR = "./codellama-c-to-cpp-finetuned"

REPO_NAME = "Berly0/CodeLlama-7b-hf-c-to-cpp"

api = HfApi()

try:
    api.create_repo(repo_id=REPO_NAME, exist_ok=True)
    print(f"Repositorio '{REPO_NAME}' creado o ya existente.")

    print(f"Subiendo archivos desde '{OUTPUT_DIR}' a '{REPO_NAME}'...")
    api.upload_folder(
        folder_path=OUTPUT_DIR,
        repo_id=REPO_NAME,
        repo_type="model",
    )
    print(f"Archivos subidos exitosamente a '{REPO_NAME}'.")

except Exception as e:
    print(f"Error al subir el modelo a Hugging Face: {e}")

Repositorio 'Berly0/CodeLlama-7b-hf-c-to-cpp' creado o ya existente.
Subiendo archivos desde './codellama-c-to-cpp-finetuned' a 'Berly0/CodeLlama-7b-hf-c-to-cpp'...


Uploading...:   0%|          | 0.00/641M [00:00<?, ?B/s]

Archivos subidos exitosamente a 'Berly0/CodeLlama-7b-hf-c-to-cpp'.


# Sistema Hibrido

In [ ]:
!sudo apt-get install g++

In [ ]:
!pip install evaluate

* LLM

In [ ]:
def load_test_dataset(test_dataset_path) -> List[Dict]:
  """Carga el dataset de prueba"""
  data = []
  with open(test_dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
      data.append(json.loads(line.strip()))
  return data

In [ ]:
def create_prompt(source_code, mode="llm"):
    """Crea el prompt para el entrenamiento"""
    if mode == "llm":
        prompt = f"""Traduce el siguiente código de C a C++:
Código C:
{source_code}

Código C++:"""
    elif mode == "llm+regex":
        prompt = f"""Traduce el siguiente código de C a C++:

Código C:
{source_code}

Código C++:"""

    return prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Berly0/CodeLlama-7b-hf-c-to-cpp"

print(f"Cargando tokenizer para {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Cargando modelo para {model_name}...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Modelo y tokenizer cargados exitosamente.")

Cargando tokenizer para Berly0/CodeLlama-7b-hf-c-to-cpp...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Modelo y tokenizer cargados exitosamente.


In [ ]:
def test_model(model, test_c_code, mode):
    """Función para probar el modelo entrenado"""

    prompt = create_prompt(test_c_code, mode)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.35,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    cpp_start_marker = "Código C++:"
    cpp_end_marker = "Ejercicio"
    cpp_start = generated_text.find(cpp_start_marker)

    if cpp_start != -1:
        cpp_start += len(cpp_start_marker)
        cpp_code = generated_text[cpp_start:].strip()

        cpp_end = cpp_code.find(cpp_end_marker)
        if cpp_end != -1:
            cpp_code = cpp_code[:cpp_end].strip()

    else:
        cpp_code = "No se encontró la sección de Código C++ en la salida."


    return cpp_code

In [ ]:
def read_text_file(file_path):
    """Lee el contenido completo de un archivo de texto."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no fue encontrado.")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")
        return None

* Regex

In [ ]:
!git clone https://github.com/Berly01/Transpilador-Compiladores.git

Cloning into 'Transpilador-Compiladores'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 2), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 11.86 KiB | 11.86 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/Transpilador-Compiladores
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (0.6s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Transpilado

In [ ]:
%cd Transpilador-Compiladores
%mkdir build

[Errno 2] No such file or directory: 'Transpilador-Compiladores'
/content/Transpilador-Compiladores
mkdir: cannot create directory ‘build’: File exists
-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Transpilador-Compiladores/build
[ 50%] Building CXX object Transpiler/CMakeFiles/Transpiler.dir/Transpiler.cpp.o
/content/Transpilador-Compiladores/Transpiler/Transpiler.cpp: In function ‘int main(int, char**)’:
/content/Transpilador-Compiladores/Transpiler/Transpiler.cpp:16:21: error: expected ‘;’ before ‘}’ token
   16 |     } else {return 1}
      |                     ^
      |                     ;
gmake[2]: *** [Transpiler/CMakeFiles/Transpiler.dir/build.make:79: Transpiler/CMakeFiles/Transpiler.dir/Transpiler.cpp.o] Error 1
gmake[1]: *** [CMakeFiles/Makefile2:106: Transpiler/CMakeFiles/Transpiler.dir/all] Error 2
gmake: *** [Makefile:91: all] Error 2


In [ ]:
!cmake -S . -B build
!cmake --build build

-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Transpilador-Compiladores/build
[ 50%] Building CXX object Transpiler/CMakeFiles/Transpiler.dir/Transpiler.cpp.o
[100%] Linking CXX executable Transpiler
[100%] Built target Transpiler


* Solo LLM

In [ ]:
file_content = read_text_file("/content/large-code.c")

In [ ]:
result = test_model(model, file_content, mode="llm")
with open("large-code-llm.cpp", "w") as f:
    f.write(result)

* Regex + LLM

In [ ]:
!/content/Transpilador-Compiladores/build/Transpiler/Transpiler /content/large-code.c /content/output.cpp

Transpilacion completada


In [ ]:
file_content = read_text_file("/content/output.cpp")

In [ ]:
result = test_model(model, file_content, mode="llm+regex")

In [ ]:
with open("/content/large-code-llm+regex.cpp", "w") as f:
    f.write(result)